In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from src import *

In [3]:
model, config = load_model_locally('clean_cross_validation_5_gaussian_blur/2', 'rvce')
transform = create_transformation(config)

In [4]:
config.gaussian_blur

5

In [5]:
datapool = DataPool(config.testing_files, config.window_length, config.split_ratio, tqdm=tqdm)

Video loading: 100%|██████████| 4/4 [00:00<00:00,  8.94it/s]


In [6]:
datapool

{'20190819-Kutna Hora-L3-out-MVI_0008': <src.video.Video at 0x7f1b84b28ac0>,
 '20190819-Kutna Hora-L18-in-MVI_0030': <src.video.Video at 0x7f1b84b28f40>,
 '20190819-Ricany-L6-in-MVI_0008': <src.video.Video at 0x7f1b84b28a00>,
 '20190819-Ricany-L9-out-MVI_0011': <src.video.Video at 0x7f1b84b28ca0>}

In [17]:
MEs = []
Ps = []
table = []

for video in tqdm(datapool):
    from_time = 0
    till_time = len(video.signal) // config.sr

    predictions = validate(video.signal, model, transform, config, classification=True)
    labels = get_labels(video.events, config.window_length, from_time, till_time)

    n_events = len(video.events)
    n_predicted = predictions.sum()
    rvce = np.abs(n_predicted - n_events) / n_events

    ME = {}
    P = {}
    for c in range(5):
        labels_eq_c = labels == c
        ME[c] = np.sum(labels_eq_c * (predictions - c)) / np.sum(labels_eq_c)
        P[c] = np.mean(labels_eq_c)

    MEs.append(ME)
    Ps.append(P)

    mae = np.abs(predictions - labels).mean()
    
    table.append([video.file, rvce, mae])
    # break

100%|██████████| 4/4 [00:06<00:00,  1.67s/it]


In [18]:
print(tabulate(table))

-----------------------------------  ---------  ---------
20190819-Kutna Hora-L3-out-MVI_0008  0.488      0.55814
20190819-Kutna Hora-L18-in-MVI_0030  0.545455   1.49419
20190819-Ricany-L6-in-MVI_0008       0.104442   0.746356
20190819-Ricany-L9-out-MVI_0011      0.0847458  0.0553936
-----------------------------------  ---------  ---------


In [25]:
for row, P, ME in zip(table, Ps, MEs):
    print(row)
    for c in P.keys():
        print(f'{c}: {P[c]:.3f}, {ME[c]:.3f}')
    print()
# for ME in MEs:
#     print([ME])

['20190819-Kutna Hora-L3-out-MVI_0008', 0.488, 0.5581395348837209]
0: 0.465, 0.469
1: 0.390, 0.440
2: 0.110, -0.132
3: 0.026, -0.444
4: 0.006, -1.000

['20190819-Kutna Hora-L18-in-MVI_0030', 0.5454545454545454, 1.494186046511628]
0: 0.102, 0.829
1: 0.241, -0.157
2: 0.314, -1.102
3: 0.183, -1.810
4: 0.099, -2.794

['20190819-Ricany-L6-in-MVI_0008', 0.10444177671068428, 0.7463556851311953]
0: 0.090, 0.484
1: 0.207, 0.113
2: 0.257, 0.023
3: 0.210, -0.347
4: 0.140, -0.646

['20190819-Ricany-L9-out-MVI_0011', 0.0847457627118644, 0.05539358600583091]
0: 0.845, 0.021
1: 0.137, -0.149
2: 0.017, -0.667
3: 0.000, nan
4: 0.000, nan



In [22]:
Ps

[{0: 0.46511627906976744,
  1: 0.38953488372093026,
  2: 0.11046511627906977,
  3: 0.02616279069767442,
  4: 0.005813953488372093},
 {0: 0.10174418604651163,
  1: 0.24127906976744187,
  2: 0.313953488372093,
  3: 0.18313953488372092,
  4: 0.09883720930232558},
 {0: 0.09037900874635568,
  1: 0.20699708454810495,
  2: 0.2565597667638484,
  3: 0.2099125364431487,
  4: 0.13994169096209913},
 {0: 0.8454810495626822,
  1: 0.13702623906705538,
  2: 0.01749271137026239,
  3: 0.0,
  4: 0.0}]

In [26]:
# from cross_validation import cross_validation_error

# uuids = []
# for i in range(5):
#     uuids.append(f'clean_cross_validation_5_small_features/{i}')
# cross_validation_error(uuids, 'mae')